In [23]:
# spark.stop()

NameError: name 'spark' is not defined

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('myApp') \
.setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc) 


24/05/22 09:58:39 WARN Utils: Your hostname, Christophers-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
24/05/22 09:58:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 09:58:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *
import pandas as pd
import math

In [3]:
df_application = spark.read.format('csv').options(header=True,inferSchema=True).load('application_record.csv')
df_application.show(30, truncate=50)
df_application.printSchema()

+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE|          NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|             Higher education|      Civil marriage| Rented apartment

24/05/21 16:21:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Check for null etc.
df_application.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_application.columns]).show()
# No null in data so all data is valid

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
| ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|  0|          0|           0|              0|           0|               0|               0|                  0|                 0|                0|         0|            0|         0|              0|         0|         0|     

In [6]:
# First Check the data in code_gender to check if there is any misdata
# Select and show distinct values for each column separately
df_application.select('code_gender').distinct().show()
df_application.select('flag_own_car').distinct().show()
df_application.select('flag_own_realty').distinct().show()
df_application.select('flag_mobil').distinct().show()
df_application.select('flag_work_phone').distinct().show()
df_application.select('flag_email').distinct().show()
df_application.select('flag_phone').distinct().show()

+-----------+
|code_gender|
+-----------+
|          F|
|          M|
+-----------+

+------------+
|flag_own_car|
+------------+
|           Y|
|           N|
+------------+

+---------------+
|flag_own_realty|
+---------------+
|              Y|
|              N|
+---------------+

+----------+
|flag_mobil|
+----------+
|         1|
+----------+

+---------------+
|flag_work_phone|
+---------------+
|              1|
|              0|
+---------------+

+----------+
|flag_email|
+----------+
|         1|
|         0|
+----------+

+----------+
|flag_phone|
+----------+
|         1|
|         0|
+----------+



In [7]:
# After checking the flag_mobil is not important because the data is only 1 and no other variation. so in the cleaned data not getting the flag mobil
df_app_cleaned = df_application.select(['id','code_gender','flag_own_car','flag_own_realty','cnt_children','amt_income_total',
                                        'name_income_type','name_education_type','name_family_status','name_housing_type','days_birth',
                                        'days_employed','flag_work_phone','flag_phone','flag_email','occupation_type','cnt_fam_members'])

# AND the value is always 0 an 1 as the boolean so we want to make our data uniform so changing the Y as 1 and N as 0, F as 1 and M as 0
df_app_cleaned = df_app_cleaned.withColumn('code_gender', when(df_app_cleaned.code_gender == 'F',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_car', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_realty', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))

# More data cleaning cnt_fam_members should be integer because there shouldn't be fam members that is half or less than 1
df_app_cleaned = df_app_cleaned.withColumn('cnt_fam_members', col('cnt_fam_members').cast('Integer'))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|              1|         0|         0|     

In [8]:
# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('age', floor(col('days_birth')/365.25) * -1)

# Check value because age should not be negative or even > 100
df_app_cleaned.select('age').where('age < 0 or age > 100').show()

# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('employed', floor(col('days_employed')/365.25) * -1)

# Check value because age should not be negative or even > 100 as well as the occupation
df_app_cleaned.select(['name_income_type','occupation_type','employed']).where('employed < 0 or employed > 100').distinct().show()

+---+
|age|
+---+
+---+

+----------------+---------------+--------+
|name_income_type|occupation_type|employed|
+----------------+---------------+--------+
|       Pensioner|           NULL|    -999|
+----------------+---------------+--------+



In [9]:
# because the data is said pensioner than it is possible that the employed is negative but for calculation sake it is better to make it into 0
df_app_cleaned = df_app_cleaned.withColumn('employed', when(df_app_cleaned.employed < 0,0).otherwise(df_app_cleaned.employed))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|    

In [12]:
# there is occupation type that is null so we need to fill it up
df_app_cleaned.select('occupation_type').where(isnull(col('occupation_type'))).count() 
# There are no possibility to

134203

In [20]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# there is possibility to use the amt_income_total & name_income_type to classified the occupation_type typically will have the same job
# so need to find the exact number to it

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
|        427500.0|         Working|           NULL|      13|
|        427500.0|         Working|           NULL|      13|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        315000.0|       Pensioner|           NULL|       0|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        157500.0|      

In [97]:
# use amt_income total to get the kind of occupation type check first
# the possibility is that the amt_income_total,name_income_type, and employed can fill the occupation_type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where((df_app_cleaned.amt_income_total == 225000.0) & (df_app_cleaned.name_income_type == 'Working')).show()
# This data further proven the point that it will be able to be filled up

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|         Working|       Managers|       3|
|        225000.0|      

In [21]:
# so use the possibility of it and fill the occupation_type that got from the amt_income_total and name_income_type
from pyspark.sql import Window, functions as F

# Define a window specification
window_spec = Window.partitionBy('amt_income_total', 'name_income_type')

# Fill missing values of occupation_type with the most frequent value within each group
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)

# Fill missing values using coalesce
df_app_cleaned = df_app_cleaned.withColumn('filled_occupation_type', filled_occupation_type)
df_app_cleaned = df_app_cleaned.withColumn('occupation_type', F.coalesce('occupation_type', 'filled_occupation_type')).drop('filled_occupation_type')

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6093713|          1|           0|              0|           0|         26100.0|       Pensioner|Secondary / secon...|             Married|House / apartment|    -21003|       365243|              0|    

In [22]:
# check for occupation type that is null  and that the amt_income_total and name_income_type same, but still null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type) & (df_app_cleaned.employed != 0)).show()
# There are no exact data to fill the null in that missing value, so the possibility is the same name_income_type with the nearest amt_income_total
# Create a window specification to partition by 'name_income_type' and order by the absolute difference in 'amt_income_total'
window_spec = Window.partitionBy('name_income_type').orderBy(F.abs(F.col('amt_income_total') - F.col('amt_income_total_with_null')))

# Create a temporary column to store the amt_income_total of rows with null occupation_type
df_temp = df_app_cleaned.withColumn('amt_income_total_with_null', F.when(F.col('occupation_type').isNull(), F.col('amt_income_total')))

# Fill missing occupation_type using the nearest amt_income_total
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)
df_temp = df_temp.withColumn('filled_occupation_type', filled_occupation_type)

# Use coalesce to fill the null values
df_app_cleaned = df_temp.withColumn('occupation_type', F.coalesce(F.col('occupation_type'), F.col('filled_occupation_type'))).drop('filled_occupation_type', 'amt_income_total_with_null')

df_app_cleaned.show()

+----------------+--------------------+---------------+--------+
|amt_income_total|    name_income_type|occupation_type|employed|
+----------------+--------------------+---------------+--------+
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38700.0|             Working|           NULL|      26|
|         41211.0|Commercial associate|           NULL|       6|
|         41211.0|Commerc

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6014233|          1|           0|              0|           1|         28723.5|Commercial associate|Secondary / secon...|           Married|House / apartment|    -14361|         -138|            

In [24]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# Data already cleaned!!

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
+----------------+----------------+---------------+--------+



In [29]:
df_credit = spark.read.format('csv').options(header=True,inferSchema=True).load('credit_record.csv')
df_credit.show(30, truncate=50)
df_credit.printSchema()
print('Status \n\
      0 => 1-29 days past due \n\
      1 => 30-59 days past due \n\
      2 => 60-89 days past due \n\
      3 => 90-119 days past due \n\
      4 => 120-149 days past due \n\
      5 => overdue/ bad debts > 150 days \n\
      C => paid successfully \n\
      X => No loan \n\
')
# The schema is already correct with id and months_balance as Integer.

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
|5001712|           -16|     0|
|5001712|           -17|     0|
|5001712|           -18|     0|
|5001713|             0|     X|
|5001713|            -1|     X|
|5001713|            -2|     X|
|5001713|            -3|     X|
|5001713|            -4|     X|
|5001713

In [30]:
# Check for null etc.
df_credit.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_credit.columns]).show()
# No null in data so all data is valid

+---+--------------+------+
| ID|MONTHS_BALANCE|STATUS|
+---+--------------+------+
|  0|             0|     0|
+---+--------------+------+



In [9]:
df_credit = df_credit.withColumn('status_class', 
                                when((df_credit['STATUS'] == '2') | 
                                    (df_credit['STATUS'] == '3') | 
                                    (df_credit['STATUS'] == '4') | 
                                    (df_credit['STATUS'] == '5'), 'Yes')
                                .otherwise(None))
df_credit.show(1000, truncate=50)

+-------+--------------+------+------------+
|     ID|MONTHS_BALANCE|STATUS|status_class|
+-------+--------------+------+------------+
|5001711|             0|     X|        NULL|
|5001711|            -1|     0|        NULL|
|5001711|            -2|     0|        NULL|
|5001711|            -3|     0|        NULL|
|5001712|             0|     C|        NULL|
|5001712|            -1|     C|        NULL|
|5001712|            -2|     C|        NULL|
|5001712|            -3|     C|        NULL|
|5001712|            -4|     C|        NULL|
|5001712|            -5|     C|        NULL|
|5001712|            -6|     C|        NULL|
|5001712|            -7|     C|        NULL|
|5001712|            -8|     C|        NULL|
|5001712|            -9|     0|        NULL|
|5001712|           -10|     0|        NULL|
|5001712|           -11|     0|        NULL|
|5001712|           -12|     0|        NULL|
|5001712|           -13|     0|        NULL|
|5001712|           -14|     0|        NULL|
|5001712| 

In [10]:
count_df = df_credit.groupBy('ID').agg(count(when(col('status_class') == 'Yes', True)).alias('status_class_count'))
count_df = count_df.withColumn('status_class', when(col('status_class_count') > 0, 'Yes').otherwise('No'))
count_df = count_df.select('ID', 'status_class')
data_join = data_join.join(count_df, on='ID', how='inner')
data_join = data_join.withColumn('target', when(col('status_class') == 'Yes', 1).otherwise(0))


In [11]:
#merged_data.filter(merged_data['target'] == 1).show()

data_join.show(20)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+------------+------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|begin_month|status_class|target|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+------------+------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working

In [12]:
status_class_counts = count_df.groupBy('status_class').count()

status_class_counts.show()

+------------+-----+
|status_class|count|
+------------+-----+
|          No|45318|
|         Yes|  667|
+------------+-----+



In [13]:
total_count = count_df.count()
status_class_proportion = status_class_counts.withColumn('proportion', col('count') / lit(total_count))
status_class_proportion.show()

+------------+-----+--------------------+
|status_class|count|          proportion|
+------------+-----+--------------------+
|          No|45318|  0.9854952701968033|
|         Yes|  667|0.014504729803196694|
+------------+-----+--------------------+



In [14]:
data_join = data_join.replace('NULL', None)
data_join= data_join.dropna()
data_join.count()

25134

In [15]:
columns = data_join.columns
# Membuat DataFrame dari daftar kolom
information = spark.createDataFrame([(col,) for col in columns], ["variable"])

# Menambahkan kolom 'IV' dengan nilai None
information = information.withColumn('IV', lit(None).cast(StringType()))

# Daftar kolom yang akan dihapus
namelist = ['FLAG_MOBIL', 'begin_month', 'dep_value', 'target', 'ID']

# Menghapus kolom-kolom yang ada dalam namelist dari information
for name in namelist:
    information = information.filter(information['variable'] != name)

# Tampilkan hasil
information.show()

+-------------------+----+
|           variable|  IV|
+-------------------+----+
|        CODE_GENDER|NULL|
|       FLAG_OWN_CAR|NULL|
|    FLAG_OWN_REALTY|NULL|
|       CNT_CHILDREN|NULL|
|   AMT_INCOME_TOTAL|NULL|
|   NAME_INCOME_TYPE|NULL|
|NAME_EDUCATION_TYPE|NULL|
| NAME_FAMILY_STATUS|NULL|
|  NAME_HOUSING_TYPE|NULL|
|         DAYS_BIRTH|NULL|
|      DAYS_EMPLOYED|NULL|
|    FLAG_WORK_PHONE|NULL|
|         FLAG_PHONE|NULL|
|         FLAG_EMAIL|NULL|
|    OCCUPATION_TYPE|NULL|
|    CNT_FAM_MEMBERS|NULL|
|       status_class|NULL|
+-------------------+----+



In [16]:
def calc_information_value(df: DataFrame, feature: str, target: str, pr: bool = False):
    # Mengisi nilai NULL
    df = df.withColumn(feature, when(col(feature).isNull(), "NULL").otherwise(col(feature)))
    
    # Menghitung jumlah total, good, dan bad untuk setiap nilai feature
    data = df.groupBy(feature).agg(
        count("*").alias("All"),
        count(when(col(target) == 0, 1)).alias("Good"),
        count(when(col(target) == 1, 1)).alias("Bad")
    )
    
    # Menghitung Share, Bad Rate, Distribution Good, dan Distribution Bad
    total_count = df.count()
    total_bad = df.filter(col(target) == 1).count()
    total_good = total_count - total_bad
    
    data = data.withColumn("Share", col("All") / total_count)
    data = data.withColumn("Bad Rate", col("Bad") / col("All"))
    data = data.withColumn("Distribution Good", (col("All") - col("Bad")) / total_good)
    data = data.withColumn("Distribution Bad", col("Bad") / total_bad)
    
    # Menghitung WoE
    data = data.withColumn("WoE", log(col("Distribution Good") / col("Distribution Bad")))
    
    # Mengganti nilai inf dan -inf dengan 0
    data = data.withColumn("WoE", when(col("WoE") == float("inf"), 0).otherwise(col("WoE")))
    data = data.withColumn("WoE", when(col("WoE") == float("-inf"), 0).otherwise(col("WoE")))
    
    # Menghitung information_value
    data = data.withColumn("information_value", col("WoE") * (col("Distribution Good") - col("Distribution Bad")))
    
    # Mengurutkan data berdasarkan feature dan value
    data = data.orderBy(feature)
    
    # Menghitung total information_value
    information_value = data.agg(sum("information_value")).collect()[0][0]
    
    if pr:
        data.show()
        print('information_value = ', information_value)
    
    print(f"This variable's information_value is: {information_value}")
    data.groupBy(feature).count().show()
    
    return information_value, data

In [17]:
def convert_dummy(df: DataFrame, feature: str, rank: int = 0) -> DataFrame:
    # Menghitung jumlah kemunculan setiap nilai pada feature
    mode = df.groupBy(feature).count().orderBy(col('count').desc()).collect()[rank][0]
    
    # Membuat dummy variables menggunakan pivot
    dummies = df.groupBy("ID").pivot(feature).agg(lit(1)).na.fill(0)
    
    # Mengganti nama kolom dummy
    for col_name in dummies.columns:
        if col_name != "ID":
            new_name = f"{feature}_{col_name}"
            dummies = dummies.withColumnRenamed(col_name, new_name)
    
    # Menghapus kolom yang paling sering muncul
    dummies = dummies.drop(f"{feature}_{mode}")
    
    # Menghapus kolom asli dari DataFrame
    df = df.drop(feature)
    
    # Menggabungkan kembali dummies dengan DataFrame asli
    df = df.join(dummies, on="ID", how="left")
    
    return df

In [18]:
def get_category(df, col, binsnum, labels, qcut=False):
    # Menyalin DataFrame asli
    localdf = df.select(col).toPandas()

    if qcut:
        # Quantile cut
        localdf['bins'] = pd.qcut(localdf[col], q=binsnum, labels=labels)
    else:
        # Equal-length cut
        localdf['bins'] = pd.cut(localdf[col], bins=binsnum, labels=labels)
    
    # Membuat DataFrame dari hasil binning
    bins_df = spark.createDataFrame(localdf[['bins']])
    
    # Menggabungkan hasil binning ke DataFrame asli
    name = f'gp_{col}'
    bins_df = bins_df.withColumnRenamed('bins', name)
    df = df.withColumn(name, bins_df[name].cast('string'))
    
    return df


In [19]:
# Mengganti nilai kolom CODE_GENDER
data_join = data_join.withColumn('CODE_GENDER', when(col('CODE_GENDER') == 'F', 0).when(col('CODE_GENDER') == 'M', 1))
# print(data_join['CODE_GENDER'].value_counts())
# Menghitung value_counts
gender_counts = data_join.groupBy('CODE_GENDER').count()
gender_counts.show()
# Menghitung information value
iv, data = calc_information_value(data_join, 'CODE_GENDER', 'target')
information = information.withColumn('IV', when(col('variable') == 'CODE_GENDER', lit(iv).cast(StringType())).otherwise(col('IV')))
data.show()



+-----------+-----+
|CODE_GENDER|count|
+-----------+-----+
|          1| 9504|
|          0|15630|
+-----------+-----+



24/05/21 00:18:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


This variable's information_value is: 0.02520350452745081


+-----------+-----+
|CODE_GENDER|count|
+-----------+-----+
|          0|    1|
|          1|    1|
+-----------+-----+



+-----------+-----+-----+---+-------------------+--------------------+-------------------+------------------+-------------------+--------------------+
|CODE_GENDER|  All| Good|Bad|              Share|            Bad Rate|  Distribution Good|  Distribution Bad|                WoE|   information_value|
+-----------+-----+-----+---+-------------------+--------------------+-------------------+------------------+-------------------+--------------------+
|          0|15630|15400|230| 0.6218667939842445| 0.01471529110684581|  0.623179022337326|0.5450236966824644|0.13400455891611937|0.010473169941325399|
|          1| 9504| 9312|192|0.37813320601575556|0.020202020202020204|0.37682097766267403|0.4549763033175355|-0.1884751226189682|0.014730334586125412|
+-----------+-----+-----+---+-------------------+--------------------+-------------------+------------------+-------------------+--------------------+

